# SAC agents

> Soft Actor Critic based agent

In [ ]:
#| default_exp agents.rl.sac

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging

# set logging level to INFO
logging.basicConfig(level=logging.INFO)

from abc import ABC, abstractmethod
from typing import Union, Optional, List, Tuple
import numpy as np
import os

from ddopnew.envs.base import BaseEnvironment
from ddopnew.agents.base import BaseAgent
from ddopnew.utils import MDPInfo, Parameter, DatasetWrapper
from ddopnew.obsprocessors import FlattenTimeDimNumpy
from ddopnew.RL_approximators import MLPStateAction, MLPActor
from ddopnew.postprocessors import ClipAction

from ddopnew.dataloaders.base import BaseDataLoader

from mushroom_rl.algorithms.actor_critic.deep_actor_critic import SAC

import torch
import torch.nn.functional as F
from torchsummary import summary

import time

In [ ]:
#| export

class RLBaseAgent(BaseAgent):

    """
    Base class for Agents that integrate MushroomRL agents.
    """

    train_mode = "env_interaction"
    
    def __init__(self, 
            environment_info: MDPInfo,
            obsprocessors: Optional[List] = None,     # default: []
            postprocessors: Optional[List] = None,     # default: []
            device: str = "cpu", # "cuda" or "cpu"
            agent_name: str | None = None
            ):

        self.device = device

        self.network_list, self.actor = self.get_network_list(return_actor=True)

        super().__init__(environment_info, obsprocessors, postprocessors, agent_name)

        self.transfer_obs_processors_to_mushroom_agent()

    def transfer_obs_processors_to_mushroom_agent(self):
    
        """ Transfer the obs-processors to the MushroomRL agent preprocessors"""

        for obsprocessor in self.obsprocessors:
            self.add_obsprocessor(obsprocessor)
        self.obsprocessors = []

    def add_obsprocessor(self, obsprocessor: object): 
        """Add an obsprocessor to the agent - overwrites the base
        class method to add the obsprocessor to the MushroomRL agent
        as preprocessor. Postprocessors stay with the base class"""
        self.agent.add_preprocessor(obsprocessor)

    @property
    def preprocessors(self):

        """ Return the preprocessors of the MushroomRL agent """

        return self.agent.preprocessors


    @abstractmethod
    def set_model(self, input_shape: Tuple, output_shape: Tuple):
        """ Set the model for the agent """
        pass

    def set_optimizer(self, optimizer_params: dict): # dict with keys: optimizer, lr, weight_decay
        
        """ Set the optimizer for the model """
        optimizer = optimizer_params["optimizer"]
        optimizer_params_copy = optimizer_params.copy()
        del optimizer_params_copy["optimizer"]

        if optimizer == "Adam":
            self.optimizer = torch.optim.Adam(self.model.parameters(), **optimizer_params_copy)
        elif optimizer == "SGD":
            self.optimizer = torch.optim.SGD(self.model.parameters(), **optimizer_params_copy)
        elif optimizer == "RMSprop":
            self.optimizer = torch.optim.RMSprop(self.model.parameters(), **optimizer_params_copy)
        else:
            raise ValueError(f"Optimizer {optimizer} not supported")

    def draw_action_(self, observation: np.ndarray) -> np.ndarray: #
        
        """ 
        Draw an action based on the fitted model (see predict method)
        """

        # Remove batch dimension if it is one
        if observation.shape[0] == 1:
            observation = observation[0]

        if self.mode=="train":
            action = self.agent.draw_action(observation)
        else:
            action = self.predict(observation) # bypass the agent's draw_action method and directly get prediction from policy network
        
        return action
        
    def predict(self, observation: np.ndarray) -> np.ndarray: #
        """ Do one forward pass of the model directly and return the prediction"""

        if self.mode=="eval":

            # Apply pre-processors of the mushroom agent
            for preprocessor in self.agent.preprocessors:
                observation = preprocessor(observation)
            
            # add batch dimension back to mimic mushroom_rl library
            observation = np.expand_dims(observation, axis=0)
            action = self.predict_(observation)

            return action
        else:
            raise ValueError("Model is in train mode. Use draw_action method instead.")

    def predict_(self, observation: np.ndarray) -> np.ndarray: #
        """ Do one forward pass of the model directly and return the prediction
        Overwrite for agents that have additional steps such as SAC"""

        observation = torch.tensor(observation, dtype=torch.float32).to(self.device)
        action = self.actor.forward(observation)
        action = action.cpu().detach().numpy()

        return action

    def train(self):
        """set the internal state of the agent and its model to train"""
        self.mode = "train"
        
        pass

    def eval(self):
        """set the internal state of the agent and its model to eval"""
        self.mode = "eval"
        
        pass

    def to(self, device: str): #
        """Move the model to the specified device"""

        # check if self.model or something else
        self.model.to(device)

    def save(self,
                path: str, # The directory where the file will be saved.
                overwrite: bool=True): # Allow overwriting; if False, a FileExistsError will be raised if the file exists.
        
        """
        Save the PyTorch model to a file in the specified directory.

        """
        
        if not hasattr(self, 'network_list') or self.network_list is None:
            raise AttributeError("Cannot find networks.")

        # Create the directory path if it does not exist
        os.makedirs(path, exist_ok=True)

        # Construct the file path using os.path.join for better cross-platform compatibility

        for network_number, network in enumerate(self.network_list):
            full_path = os.path.join(path, f"network_{network_number}.pth")

            if os.path.exists(full_path):
                if not overwrite:
                    raise FileExistsError(f"The file {full_path} already exists and will not be overwritten.")
                else:
                    logging.debug(f"Overwriting file {full_path}") # Only log with info as during training we will continuously overwrite the model
            
            # Save the model's state_dict using torch.save
            torch.save(network.state_dict(), full_path)
        logging.debug(f"Model saved successfully to {full_path}")


    def load(self, path: str): # Only the path to the folder is needed, not the file itself
 
        """
        Load the PyTorch model from a file.
        """
        
        if not hasattr(self, 'model') or self.model is None:
            raise AttributeError("Model is not defined in the class.")

        # Construct the file path
        full_path = os.path.join(path, "model.pth")

        if not os.path.exists(full_path):
            raise FileNotFoundError(f"The file {full_path} does not exist.")

        try:
            # Load the model's state_dict using torch.load
            self.model.load_state_dict(torch.load(full_path))
            logging.info(f"Model loaded successfully from {full_path}")
        except Exception as e:
            raise RuntimeError(f"An error occurred while loading the model: {e}")

    def set_device(self, device: str):

        """ Set the device for the model """

        if device == "cuda":
            if torch.cuda.is_available():
                use_cuda = True
            else:
                logging.warning("CUDA is not available. Using CPU instead.")
                use_cuda = False
        elif device == "cpu":
            use_cuda = False
        else:
            raise ValueError(f"Device {device} not currently not supported, use 'cuda' or 'cpu'")

        return use_cuda

    @staticmethod
    def get_optimizer_class(optimizer_name: str): #

        """ Get optimizer class based on the optimizer name """

        if optimizer_name == "Adam":
            return torch.optim.Adam
        elif optimizer_name == "SGD":
            return torch.optim.SGD
        elif optimizer_name == "RMSprop":
            return torch.optim.RMSprop
        else:
            raise ValueError(f"Optimizer {optimizer_name} not supported")

    @staticmethod
    def get_loss_function(loss: str): #

        """ Get optimizer class based on the optimizer name """

        if loss == "MSE":
            return F.mse_loss
        else:
            raise ValueError(f"Loss {loss} not supported")

    @staticmethod
    def get_input_shape(observation_space: object, flatten_time_dim: bool = True): #

        """ Get the input shape of the model based on the environment info """

        # TODO: Account for more complex spaces like dicts

        observation_space_shape = observation_space.shape

        if flatten_time_dim:
            input_shape = (np.prod(observation_space_shape),)
        else:
            input_shape = input_shape

        return input_shape

    def episode_start(self):

        """ What to do if a new episode starts, e.g., reset policy of the agent
        Often this does not need to do anything (default), otherwise this funciton 
        needs to be overwritten in the subclass. """

        pass

    def fit(self, dataset, **dataset_info):

        """ Hand the fit mehtod to the mushroom agent """

        self.agent.fit(dataset, **dataset_info)

    def stop(self):
        """ Stop the agent """

        self.agent.stop()

In [ ]:
show_doc(RLBaseAgent, title_level=2)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/agents/rl/sac.py#L35){target="_blank" style="float:right; font-size:smaller"}

## RLBaseAgent

>      RLBaseAgent (environment_info:ddopnew.utils.MDPInfo,
>                   obsprocessors:Optional[List]=None,
>                   postprocessors:Optional[List]=None, device:str='cpu',
>                   agent_name:str|None=None)

*Base class for Agents that integrate MushroomRL agents.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| environment_info | MDPInfo |  |  |
| obsprocessors | Optional | None | default: [] |
| postprocessors | Optional | None | default: [] |
| device | str | cpu | "cuda" or "cpu" |
| agent_name | str \| None | None |  |

### XXX

XXX

**XXXs**:

* XXX

In [ ]:
#| export

class SACAgent(RLBaseAgent):

    """
    XXX
    """

    dropout = True # always keep in True for mushroom_RL, dropout is not desired set drop_prob=0.0

    def __init__(self, 
                environment_info: MDPInfo,

                learning_rate_actor: float = 3e-4,
                learning_rate_critic: float | None = None, # If none, then it is set to learning_rate_actor
                initial_replay_size: int = 64,
                max_replay_size: int = 50000,
                batch_size: int = 64,
                hidden_layers: List = None, # if None, then default is [64, 64]
                activation: str = "relu", # "relu", "sigmoid", "tanh", "leakyrelu", "elu"
                warmup_transitions: int = 100,
                lr_alpha: float = 3e-4,
                tau: float = 0.005,
                log_std_min: float = -20.0,
                log_std_max: float = 2.0,
                use_log_alpha_loss=False,
                target_entropy: float | None = None,
                optimizer: str = "Adam", # "Adam" or "SGD" or "RMSprop"  
                loss: str = "MSE", # currently only MSE is supported     
            
                obsprocessors: list | None = None,      # default: []
                postprocessors: list | None = None,     # default: []
                device: str = "cpu", # "cuda" or "cpu"
                agent_name: str | None = "SAC",
                ):

        # The standard SAC agent needs a 2D input, so we need to flatten the time dimension
        flatten_time_dim_processor = FlattenTimeDimNumpy(allow_2d=True, batch_dim_included=False)
        obsprocessors = (obsprocessors or []) + [flatten_time_dim_processor]

        use_cuda = self.set_device(device)

        hidden_layers = hidden_layers or [64, 64]
        self.warmup_training_steps = initial_replay_size


        OptimizerClass=self.get_optimizer_class(optimizer)
        learning_rate_critic = learning_rate_critic or learning_rate_actor
        lossfunction = self.get_loss_function(loss)

        actor_input_shape = self.get_input_shape(environment_info.observation_space)
        actor_output_shape = environment_info.action_space.shape
        critic_input_shape = (actor_input_shape[0] + actor_output_shape[0],) # check how this works when RNN and mixed agents are used

        actor_mu_params = dict(network=MLPActor,
                                hidden_layers=hidden_layers,
                                input_shape=actor_input_shape,
                                output_shape=actor_output_shape,
                                activation=activation,
                                use_cuda=use_cuda,
                                dropout=self.dropout
                                )

        actor_sigma_params = dict(network=MLPActor,
                                    hidden_layers=hidden_layers,
                                    input_shape= actor_input_shape,
                                    output_shape=actor_output_shape,
                                    activation=activation,
                                    use_cuda=use_cuda,
                                    dropout=self.dropout 
                                    )
        
        actor_optimizer = {'class': OptimizerClass,
            'params': {'lr': learning_rate_actor}} 

        critic_params = dict(network=MLPStateAction,
                optimizer={'class': OptimizerClass,
                        'params': {'lr': learning_rate_critic}}, 
                loss=lossfunction,
                # n_features=n_features,
                input_shape=critic_input_shape,
                hidden_layers=hidden_layers,
                output_shape=(1,),
                # squeeze_output=squeeze_output,
                use_cuda=use_cuda,
                dropout=self.dropout,)

        self.agent = SAC(
            mdp_info=environment_info,
            actor_mu_params=actor_mu_params,
            actor_sigma_params=actor_sigma_params,
            actor_optimizer=actor_optimizer,
            critic_params=critic_params,
            batch_size=batch_size,
            initial_replay_size=initial_replay_size,
            max_replay_size=max_replay_size,
            warmup_transitions=warmup_transitions,
            tau=tau,
            lr_alpha=lr_alpha,
            use_log_alpha_loss=use_log_alpha_loss,
            log_std_min=log_std_min,
            log_std_max=log_std_max,
            target_entropy=target_entropy,
            critic_fit_params=None
        )

        all_attributes_methods = dir(self.agent)

        # Filter out special attributes/methods (those starting and ending with double underscores)
        filtered_attributes_methods = [item for item in all_attributes_methods if not item.startswith('__')]

        super().__init__(environment_info, obsprocessors, postprocessors, device, agent_name)

        logging.info("Actor network:")
        if logging.getLogger().isEnabledFor(logging.INFO):
            summary(self.actor, input_size=actor_input_shape)

    def get_network_list(self, return_actor: bool = True):
        """ Get the list of networks in the agent for the save and load functions
        Get the actor for the predict function in eval mode """

        networks = []
        ensemble_critic = self.agent._critic_approximator._impl.model
        for i, model in enumerate(ensemble_critic):
            networks.append(model.network)
        networks.append(self.agent.policy._mu_approximator._impl.model.network)
        networks.append(self.agent.policy._sigma_approximator._impl.model.network)

        actor = self.agent.policy._mu_approximator._impl.model.network
        
        if return_actor:
            return networks, actor
        else:
            return networks
    
    def load(self, path: str):
            pass

    def predict_(self, observation: np.ndarray) -> np.ndarray: #
        """ Do one forward pass of the model directly and return the prediction.
        Apply tanh as implemented for the SAC actor in mushroom_rl"""

        # make observation torch tensor

        observation = torch.tensor(observation, dtype=torch.float32).to(self.device)
        action = self.actor.forward(observation)
        # print("a before tanh: ", action)
        action = torch.tanh(action)
        # print("a after tanh: ", action)
        action = action * self.agent.policy._delta_a + self.agent.policy._central_a
        # print("a after scaling: ", action)
        action = action.cpu().detach().numpy()

        return action

In [ ]:
from ddopnew.envs.inventory import NewsvendorEnv
from ddopnew.dataloaders.tabular import XYDataLoader
from ddopnew.experiment_functions import run_experiment, test_agent

val_index_start = 8000 #90_000
test_index_start = 9000 #100_000

X = np.random.standard_normal((10000, 2))
Y = np.random.standard_normal((10000, 1))
Y += 2*X[:,0].reshape(-1, 1) + 3*X[:,1].reshape(-1, 1)
Y = X[:,0].reshape(-1, 1)
# truncate Y at 0:
Y = np.maximum(Y, 0)
# normalize Y max to 1
Y = Y/np.max(Y)

print(np.max(Y))

print(X.shape, Y.shape)

dataloader = XYDataLoader(X, Y, val_index_start, test_index_start, lag_window_params =  {'lag_window': 0, 'include_y': False, 'pre_calc': True})

environment = NewsvendorEnv(
    dataloader = dataloader,
    underage_cost = 0.42857,
    overage_cost = 1.0,
    gamma = 0.999,
    horizon_train = 365,
    q_bound_high = 1.0,
    q_bound_low = -0.1
)

clip_action = ClipAction(0., 1.)

agent = SACAgent(environment.mdp_info,
                obsprocessors = None,      # default: []
                postprocessors = [clip_action],     # default: []
                device="cpu", # "cuda" or "cpu"
)

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

environment.train()
agent.train()
environment.print=False

# run_experiment(agent, environment, n_epochs=50, n_steps=1000, run_id = "test", save_best=True, print_freq=1) # fit agent via run_experiment function

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

1.0
(10000, 2) (10000, 1)
reset to: 4774
weights of first layer critic: #####################################
tensor([-0.3942,  0.3179,  0.3138], grad_fn=<SliceBackward0>)


/Users/magnus/miniforge3/envs/inventory_gym_2/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


weights of first layer critic: #####################################
tensor([-0.3942,  0.3179,  0.3138], grad_fn=<SliceBackward0>)
weights of first layer critic: #####################################
tensor([-0.3942,  0.3179,  0.3138], grad_fn=<SliceBackward0>)
weights of first layer critic: #####################################
tensor([-0.3942,  0.3179,  0.3138], grad_fn=<SliceBackward0>)
weights of first layer: #####################################
tensor([-0.4002, -0.2924], grad_fn=<SliceBackward0>)
weights of first layer: #####################################
tensor([-0.4002, -0.2924], grad_fn=<SliceBackward0>)
state torch.Size([2, 2])
tensor([0.3906, 0.0000, 0.3843, 0.3568, 0.0000, 0.2708, 0.0000, 0.0000, 0.0000,
        0.0000], grad_fn=<SliceBackward0>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             192
            Linear-2                

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()